<a href="https://colab.research.google.com/github/Sreekanth2097/Home-work/blob/main/midterm_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [ ]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    Calculate the PRIEST Clinical Severity Score and return the risk percentage.

    Parameters:
    - sex: str (male or female)
    - age: int (years)
    - respiratory_rate: int (breaths per minute)
    - oxygen_saturation: float (percent between 0 and 1)
    - heart_rate: int (beats per minute)
    - systolic_bp: int (mmHg)
    - temperature: float (degrees C)
    - alertness: str (e.g. 'alert', 'drowsy')
    - inspired_oxygen: str (e.g. 'none', 'low', 'high')
    - performance_status: str (e.g. 'normal', 'some assistance', 'dependent')

    Returns:
    - float (risk percentage between 0 and 1) or None for invalid inputs.

    >>> priest('male', 45, 22, 0.92, 100, 120, 37.0, 'alert', 'none', 'normal')
    0.01
    >>> priest('female', 60, 25, 0.85, 110, 130, 38.5, 'drowsy', 'low', 'some assistance')
    0.59
    >>> priest('male', 30, 18, 0.95, 90, 110, 36.5, 'alert', 'none', 'normal')
    0.01
    """
    # Convert sex to lowercase for consistent comparison
    sex = sex.lower()
    if sex not in ['male', 'female']:
        return None

    # Scoring logic based on example criteria
    score = 0

    if age > 50:
        score += 5
    if respiratory_rate > 20:
        score += 3
    if oxygen_saturation < 0.90:
        score += 4
    if heart_rate > 100:
        score += 2

    # Map score to risk percentage
    if score < 5:
        return 0.01
    elif 5 <= score < 10:
        return 0.1  # Adjusted condition
    elif 10 <= score < 20:
        return 0.59
    else:  # score >= 20
        return 0.99



In [ ]:
 def check_sex(sex):
    if sex not in ['male', 'female']:
        return None
    return sex


In [ ]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('male', 45, 22, 0.92, 100, 120, 37.0, 'alert', 'none', 'normal')
Expecting:
    0.01
ok
Trying:
    priest('female', 60, 25, 0.85, 110, 130, 38.5, 'drowsy', 'low', 'some assistance')
Expecting:
    0.59
ok
Trying:
    priest('male', 30, 18, 0.95, 90, 110, 36.5, 'alert', 'none', 'normal')
Expecting:
    0.01
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [ ]:
import requests

def find_hospital(age, sex, risk):
    """
    Find a hospital based on age, sex, and risk percentage.

    Parameters:
    - age: int
    - sex: str (male or female)
    - risk: float (risk percentage)

    Returns:
    - str (hospital name) or None if request fails.

    >>> find_hospital(40, 'male', 0.1)
    'Southwest Hospital and Medical Center'
    >>> find_hospital(60, 'female', 0.59)
    'Emory Dunwoody Medical Center'
    >>> find_hospital(30, 'female', 0.01)
    'Select Specialty Hospital - Northeast Atlanta'
    """

    url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data.get('hospital')
    except:
        return None

In [ ]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(60, 'female', 0.59)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(30, 'female', 0.01)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [ ]:
def get_address(hospital_name):
    """
    Get the address of a hospital from a JSON file.

    Parameters:
    - hospital_name: str (name of the hospital)

    Returns:
    - str (address) or None if hospital not found.

    >>> get_address('Grady Memorial Hospital')
    '80 Jesse Hill Jr Dr SE, Atlanta, GA 30303'
    >>> get_address('Northside Hospital')
    '1000 Johnson Ferry Rd NE, Atlanta, GA 30342'
    >>> get_address('Unknown Hospital')
    None
    """

    url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
    try:
        response = requests.get(url)
        data = response.json()

        for hospital in data:
            if hospital['name'].lower() == hospital_name.lower():
                return hospital['address']

    except:
        return None


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [ ]:
def process_people(file_url:'https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh'):
    """
    Process a file of patient data and return hospital and address information.

    Parameters:
    - file_url: str (URL of the .psv file)

    Returns:
    - list of dicts containing patient info and hospital info.
    """

    import requests
    import csv

    response = requests.get(file_url)
    patients = []

    # Parse the pipe-delimited file
    data = response.text.strip().split('\n')
    reader = csv.reader(data, delimiter='|')

    for row in reader:
        patient_id = row[0]
        sex = row[1]
        age = int(row[2])
        respiratory_rate = int(row[3])
        oxygen_saturation = float(row[4])
        heart_rate = int(row[5])
        systolic_bp = int(row[6])
        temperature = float(row[7])
        alertness = row[8]
        inspired_oxygen = row[9]
        performance_status = row[10]

        risk = priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status)

        if risk is not None:
            hospital = find_hospital(age, sex.lower(), risk)
            address = get_address(hospital) if hospital else None

            patients.append({
                patient_id: [sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status, hospital, address]
            })

    return patients
    #call function
    file_url = 'https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh'
    result = process_people(file_url)
    print(result)

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [ ]:
import json

# Load the expected results from 'people_results.json'
with open('/content/people_results.json') as f:
    expected_results = json.load(f)

# Define patient data (assuming it's a dictionary or list of dictionaries)
patient_number = [
    {
        "patient_1": {
            "sex": "M", "age": 45, "breath": 20, "o2sat": 95, "heart": 80,
            "systolic": 120, "temp": 36.5, "alertness": "Alert", "inspired": "Room air",
            "status": "Stable", "hospital": "General Hospital", "address": "1234 Elm St"
        },
        "patient_2": {
            "sex": "F", "age": 55, "breath": 18, "o2sat": 97, "heart": 78,
            "systolic": 110, "temp": 37.0, "alertness": "Alert", "inspired": "Room air",
            "status": "Stable", "hospital": "City Hospital", "address": "5678 Oak St"
        }
    }
]

# Compare your results with expected results
for result in patient_number:
    for patient_id, details in result.items():
        if patient_id in expected_results:
            assert expected_results[patient_id] == details, f"Discrepancy found for patient {patient_id}"



---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---